## **1. 原理与算法**

**系统要素**
1、两条移动平均线
2、RSI指标
3、唐其安通道

**入场条件**
1、短期均线在长期均线之上、RSI低于超买值、创新高，则开多单(买入)
2、短期均线在长期均线之下、RSI高于超卖值、创新低，则开空单(卖出)

**入场条件**
1、持有多单时小于唐奇安通道下轨，平多单(卖出)
2、持有空单时大于唐奇安通道上轨，平空单(买入)

In [2]:
# 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。
from vitu import ai, log
import numpy as np

# 配置账户初始持仓信息
ai.create_account(name='account1', exchange='binance', account_type='digital.spot',
                  position_base=[{'asset': 'BTC', 'qty': 10}, {'asset': 'USDT', 'qty': 100000}])

# 在这个方法中编写任何的初始化逻辑，context对象将会在你的算法策略的任何方法之间做传递
def initialize(context):
    # 设置这个策略当中会用到的参数，在策略中可以随时调用
    context.account = context.get_account('account1')
    context.buy_order = list()
    context.sell_order = list()
    context.max_order = 10
    context.qty = 0.2
    context.short = 3  # 短均线周期
    context.long = 21  # 长均线周期
    context.length = 9  # RSI参数
    context.oversold = 30  # 超卖
    context.overbought = 70  # 超买
    context.donchain = 21  # 唐其安通道参数

# 你选择的universe crypto的数据更新将会触发此段逻辑，例如日线历史数据或者是实时数据更新
def handle_data(context):
    # 开始编写你的主要的算法逻辑
    data = context.history('BTC/USDT.binance', ['close','high','low'], bars=context.long, rtype='ndarray')
    close_price = data.T[0]
    high_price = data.T[1]
    low_price = data.T[2]
    # 计算长短均线
    MA_short = np.mean(close_price[-context.short:])
    MA_long = np.mean(close_price[-context.long:])
    # 计算RSI值
    rsi_price = close_price[-context.length:]
    RS_up = sum([max(rsi_price[i+1]-rsi_price[i], 0) for i in range(len(rsi_price)-1)])
    RS_all = sum([abs(rsi_price[i+1]-rsi_price[i]) for i in range(len(rsi_price)-1)])
    RS = RS_up / RS_all
    RSI = (100*RS)/(1+RS)
    # 唐奇安通道上轨
    donchain_up = np.max(high_price[-context.donchain:])
    # 唐奇安通道下轨
    donchain_down = np.min(low_price[-context.donchain:])

    # 获取最新价格
    current_price = context.get_price("BTC/USDT.binance")
    # 短期均线在长期均线之上，RSI处于超买值，创新高，则开多单(买入)
    if len(context.buy_order) < context.max_order and MA_short > MA_long and RSI < context.overbought and high_price[-1]>high_price[-2]:
        context.account.buy("BTC/USDT.binance", current_price, context.qty)
        context.buy_order.append([current_price,context.qty])
    # 短期均线在长期均线之下、RSI高于超卖值、创新低，则开空单(卖出)
    if len(context.sell_order) < context.max_order and MA_short < MA_long and RSI > context.oversold and low_price[-1]<low_price[-2]:
        context.account.sell("BTC/USDT.binance", current_price, context.qty)
        context.sell_order.append([current_price,context.qty])
    # 持有多单时小于唐奇安通道下轨，平多单(卖出)
    if len(context.buy_order) > 0 and current_price < donchain_up:
        qty = context.buy_order[-1][1]
        context.buy_order.pop()
        context.account.sell("BTC/USDT.binance", current_price, qty)
    # 持有空单时大于唐奇安通道下轨，平空单(买入)
    if len(context.sell_order) > 0 and current_price > donchain_down:
        qty = context.sell_order[-1][1]
        context.sell_order.pop()
        context.account.buy("BTC/USDT.binance", current_price, qty)

# 可以直接指定universe，或者通过筛选条件选择universe池
universe = ai.create_universe(['BTC/USDT.binance'])

# 配置策略参数如：基准、回测数据级别等
my_strategy = ai.create_strategy(
    initialize,
    handle_data,
    universe=universe,
    benchmark='csi5',
    freq='d',
    refresh_rate=1,
)

# 配置回测参数如：回测日期、手续费率
ai.backtest(
    strategy=my_strategy,
    start='2018-10-10',
    end='2019-10-10',
    commission={'taker': 0.0002, 'maker': 0.0002}
)

/opt/data/vitu/bundle/binance/1d/btcusdt


SymbolNotSupported: 交易所binance频率为1d的交易对btcusdt不在支持范围内